# Name : Aayush Modi
### Hand Gesture Recognition Model
### Task 4 : Develop a hand gesture recognition model that can accurately identify and classify different hand gestures from image or video data, enabling intuitive human-computer interaction and gesture-based control systems.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import cv2
import numpy as np

In [2]:
# Data Augmentation
data_gen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [3]:
try:
    # Data Generators
    train_generator = data_gen.flow_from_directory(
        'C:/Users/dellf/Downloads/SkillCraft Intern/Task 4/data/leapGestRecog',  # Update this path to your dataset
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        subset='training',
        seed=42  # Set random seed for reproducibility
    )

    validation_generator = data_gen.flow_from_directory(
        'C:/Users/dellf/Downloads/SkillCraft Intern/Task 4/data/leapGestRecog',  # Update this path to your dataset
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical',
        subset='validation',
        seed=42  # Set random seed for reproducibility
    )

    # Print generator values for debugging
    print("Train Generator:", train_generator)
    print("Validation Generator:", validation_generator)

    # Check if generators are None before using them
    if train_generator is None or validation_generator is None:
        print("One or both generators are None. Please check the data path and generator creation.")
        exit()

except Exception as e:  # Catch any potential errors during data generation
    print("Error occurred while creating data generators:", e)
    exit()  # Terminate the program if data generation fails

Found 16000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.
Train Generator: <keras.src.legacy.preprocessing.image.DirectoryIterator object at 0x000002ED07B46C00>
Validation Generator: <keras.src.legacy.preprocessing.image.DirectoryIterator object at 0x000002ED2351C9B0>


In [4]:
# Model Building
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes,activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy']) 

c:\Users\dellf\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Model Training with Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)  # Early stopping to prevent overfitting

history = model.fit(
    train_generator,
    epoch=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping]
)

500/500 ━━━━━━━━━━━━━━━━━━━━ 140s 279ms/step - accuracy: 0.5744 - loss: 1.1668 - val_accuracy: 0.6603 - val_loss: 0.9588


In [7]:
# Print training history for debugging
print("Training History:")
print(f"Loss: {history.history['loss']}")
print(f"Accuracy: {history.history['accuracy']}")
print(f"Validation Loss: {history.history['val_loss']}")
print(f"Validation Accuracy: {history.history['val_accuracy']}")

Training History:
Loss: [0.9845305681228638]
Accuracy: [0.6464375257492065]
Validation Loss: [0.9587534666061401]
Validation Accuracy: [0.6602500081062317]


In [8]:
# Model Evaluation
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 140ms/step - accuracy: 0.6457 - loss: 1.0002
Validation Accuracy: 65.15%


In [9]:
# Save the model
model.save('hand_gesture_recognition_model.h5')

# Load the model
loaded_model = tf.keras.models.load_model('hand_gesture_recognition_model.h5')

In [11]:
#real time prediction
import cv2
import numpy as np
import tensorflow as tf
capture = cv2.VideoCapture(0)
while True:
    ret, frame = capture.read()
    if not ret:
        break
    img = cv2.resize(frame, (128, 128))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    loaded_model = tf.keras.models.load_model('hand_gesture_recognition_model.h5')

    prediction = loaded_model.predict(img)
    gesture = np.argmax(prediction, axis=1)
    
    cv2.putText(frame, f'Gesture: {gesture}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Hand Gesture Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


KeyboardInterrupt: 

: 